# Cost and scaling model

WISDEM offers a couple different levels of fidelity for estimating mass, cost, and LCOE.  The simplest form is an implementation of a "spreadsheet" type model, where given a few high-level parameters, such as rotor diameter and machine rating, the user can use a series of empirical relationships (regression-based) to estimate turbine mass and cost.  This cost and scaling model is demonstrated in this example.

In [2]:
from wisdem.turbine_costsse.nrel_csm_tcc_2015 import nrel_csm_mass_2015, nrel_csm_2015
from openmdao.api import Problem
import numpy as np

## Mass model

The goal of this model is too model the mass given a set of parameters about the turbine.

The key to accomplishing this is the `nrel_csm_mass_2015` class.

Here, we will do a model based on this class.

First, we instantiate the `nrel_csm_mass_2015` class. Then we make an OpenMDAO `Problem` that uses this class. Then we set the inputs for our model.

In [3]:
turbine = nrel_csm_mass_2015()
prob = Problem(turbine)
prob.setup()

prob['rotor_diameter'] = 126.0
prob['turbine_class'] = 1
prob['blade_has_carbon'] = False
prob['blade_number'] = 3    
prob['machine_rating'] = 5000.0
prob['hub_height'] = 90.0
prob['bearing_number'] = 2
prob['crane'] = True

Some of the inputs require a bit more calculation before we input them into the model, such as the rotor force inputs. Here's an example where we use max tip speed, machine rating, max efficiency, and rotor speed to set the rotor torque.

In [4]:
# Rotor force calculations for nacelle inputs
maxTipSpd = 80.0
maxEfficiency = 0.90

ratedHubPower  = prob['machine_rating']*1000. / maxEfficiency 
rotorSpeed     = (maxTipSpd/(0.5*prob['rotor_diameter'])) * (60.0 / (2*np.pi))
prob['rotor_torque'] = ratedHubPower/(rotorSpeed*(np.pi/30))

Now we are set to run the model:

In [8]:
prob.run_driver()

False

Now list the model inputs

In [9]:
prob.model.list_inputs(units=True)

71 Input(s) in 'model'
----------------------

varname                           value              units
--------------------------------  -----------------  -----
top
  blade
    rotor_diameter                [126.]             m    
    blade_mass_coeff              [0.5]              None 
    blade_user_exp                [2.5]              None 
    turbine_class                 1                       
    blade_has_carbon              False                   
  hub
    blade_mass                    [18590.66820649]   kg   
    hub_mass_coeff                [2.3]              None 
    hub_mass_intercept            [1320.]            None 
  pitch
    blade_mass                    [18590.66820649]   kg   
    pitch_bearing_mass_coeff      [0.1295]           None 
    pitch_bearing_mass_intercept  [491.31]           None 
    bearing_housing_percent       [0.328]            None 
    mass_sys_offset               [555.]             None 
    blade_number                  3       

[('blade.rotor_diameter', {'value': array([126.]), 'units': 'm'}),
 ('blade.blade_mass_coeff', {'value': array([0.5]), 'units': None}),
 ('blade.blade_user_exp', {'value': array([2.5]), 'units': None}),
 ('hub.blade_mass', {'value': array([18590.66820649]), 'units': 'kg'}),
 ('hub.hub_mass_coeff', {'value': array([2.3]), 'units': None}),
 ('hub.hub_mass_intercept', {'value': array([1320.]), 'units': None}),
 ('pitch.blade_mass', {'value': array([18590.66820649]), 'units': 'kg'}),
 ('pitch.pitch_bearing_mass_coeff', {'value': array([0.1295]), 'units': None}),
 ('pitch.pitch_bearing_mass_intercept',
  {'value': array([491.31]), 'units': None}),
 ('pitch.bearing_housing_percent', {'value': array([0.328]), 'units': None}),
 ('pitch.mass_sys_offset', {'value': array([555.]), 'units': None}),
 ('spinner.rotor_diameter', {'value': array([126.]), 'units': 'm'}),
 ('spinner.spinner_mass_coeff', {'value': array([15.5]), 'units': None}),
 ('spinner.spinner_mass_intercept', {'value': array([-980.]

Now list the model outputs

In [11]:
prob.model.list_outputs(units=True)

24 Explicit Output(s) in 'model'
--------------------------------

varname                value              units
---------------------  -----------------  -----
top
  sharedIndeps
    machine_rating     [5000.]            kW   
    rotor_diameter     [126.]             m    
    blade_number       3                       
    crane              True                    
  blade
    blade_mass         [18590.66820649]   kg   
  hub
    hub_mass           [44078.53687493]   kg   
  pitch
    pitch_system_mass  [10798.90594644]   kg   
  spinner
    spinner_mass       [973.]             kg   
  lss
    lss_mass           [22820.27928238]   kg   
  bearing
    main_bearing_mass  [2245.41649102]    kg   
  gearbox
    gearbox_mass       [43468.32086769]   kg   
  hss
    hss_mass           [994.7]            kg   
  generator
    generator_mass     [14900.]           kg   
  bedplate
    bedplate_mass      [41765.26095285]   kg   
  yaw
    yaw_mass           [12329.96247921]   kg   
  hva

[('sharedIndeps.machine_rating', {'value': array([5000.]), 'units': 'kW'}),
 ('sharedIndeps.rotor_diameter', {'value': array([126.]), 'units': 'm'}),
 ('blade.blade_mass', {'value': array([18590.66820649]), 'units': 'kg'}),
 ('hub.hub_mass', {'value': array([44078.53687493]), 'units': 'kg'}),
 ('pitch.pitch_system_mass',
  {'value': array([10798.90594644]), 'units': 'kg'}),
 ('spinner.spinner_mass', {'value': array([973.]), 'units': 'kg'}),
 ('lss.lss_mass', {'value': array([22820.27928238]), 'units': 'kg'}),
 ('bearing.main_bearing_mass',
  {'value': array([2245.41649102]), 'units': 'kg'}),
 ('gearbox.gearbox_mass', {'value': array([43468.32086769]), 'units': 'kg'}),
 ('hss.hss_mass', {'value': array([994.7]), 'units': 'kg'}),
 ('generator.generator_mass', {'value': array([14900.]), 'units': 'kg'}),
 ('bedplate.bedplate_mass', {'value': array([41765.26095285]), 'units': 'kg'}),
 ('yaw.yaw_mass', {'value': array([12329.96247921]), 'units': 'kg'}),
 ('hvac.hvac_mass', {'value': array([4

# Cost model

Now that we know the mass, how much could our turbine system cost? Using the `nrel_csm_2015` with our OpenMDAO `Problem`, we can model the cost. We set up our model inputs in a similar way that we did before:

In [13]:
# simple test of module
turbine = nrel_csm_2015()
prob = Problem(turbine)
prob.setup()

# simple test of module
prob['rotor_diameter'] = 126.0
prob['turbine_class'] = 1
prob['blade_has_carbon'] = False
prob['blade_number'] = 3    
prob['machine_rating'] = 5000.0
prob['hub_height'] = 90.0
prob['bearing_number'] = 2
prob['crane'] = True

# Rotor force calculations for nacelle inputs
maxTipSpd = 80.0
maxEfficiency = 0.90

ratedHubPower  = prob['machine_rating']*1000. / maxEfficiency 
rotorSpeed     = (maxTipSpd/(0.5*prob['rotor_diameter'])) * (60.0 / (2*np.pi))
prob['rotor_torque'] = ratedHubPower/(rotorSpeed*(np.pi/30))

prob.run_driver()

False

And list our inputs and outputs as before:

In [15]:
prob.model.list_inputs(units=True)

214 Input(s) in 'model'
-----------------------

varname                                      value               units 
-------------------------------------------  ------------------  ------
top
  nrel_csm_mass
    blade
      rotor_diameter                         [126.]              m     
      blade_mass_coeff                       [0.5]               None  
      blade_user_exp                         [2.5]               None  
    hub
      blade_mass                             [18590.66820649]    kg    
      hub_mass_coeff                         [2.3]               None  
      hub_mass_intercept                     [1320.]             None  
    pitch
      blade_mass                             [18590.66820649]    kg    
      pitch_bearing_mass_coeff               [0.1295]            None  
      pitch_bearing_mass_intercept           [491.31]            None  
      bearing_housing_percent                [0.328]             None  
      mass_sys_offset                  

[('nrel_csm_mass.blade.rotor_diameter',
  {'value': array([126.]), 'units': 'm'}),
 ('nrel_csm_mass.blade.blade_mass_coeff',
  {'value': array([0.5]), 'units': None}),
 ('nrel_csm_mass.blade.blade_user_exp',
  {'value': array([2.5]), 'units': None}),
 ('nrel_csm_mass.hub.blade_mass',
  {'value': array([18590.66820649]), 'units': 'kg'}),
 ('nrel_csm_mass.hub.hub_mass_coeff', {'value': array([2.3]), 'units': None}),
 ('nrel_csm_mass.hub.hub_mass_intercept',
  {'value': array([1320.]), 'units': None}),
 ('nrel_csm_mass.pitch.blade_mass',
  {'value': array([18590.66820649]), 'units': 'kg'}),
 ('nrel_csm_mass.pitch.pitch_bearing_mass_coeff',
  {'value': array([0.1295]), 'units': None}),
 ('nrel_csm_mass.pitch.pitch_bearing_mass_intercept',
  {'value': array([491.31]), 'units': None}),
 ('nrel_csm_mass.pitch.bearing_housing_percent',
  {'value': array([0.328]), 'units': None}),
 ('nrel_csm_mass.pitch.mass_sys_offset',
  {'value': array([555.]), 'units': None}),
 ('nrel_csm_mass.spinner.rotor

In [17]:
prob.model.list_outputs(units=True)

90 Explicit Output(s) in 'model'
--------------------------------

varname                                      value               units 
-------------------------------------------  ------------------  ------
top
  nrel_csm_mass
    sharedIndeps
      machine_rating                         [5000.]             kW    
      rotor_diameter                         [126.]              m     
    blade
      blade_mass                             [18590.66820649]    kg    
    hub
      hub_mass                               [44078.53687493]    kg    
    pitch
      pitch_system_mass                      [10798.90594644]    kg    
    spinner
      spinner_mass                           [973.]              kg    
    lss
      lss_mass                               [22820.27928238]    kg    
    bearing
      main_bearing_mass                      [2245.41649102]     kg    
    gearbox
      gearbox_mass                           [43468.32086769]    kg    
    hss
      hss_mass          

[('nrel_csm_mass.sharedIndeps.machine_rating',
  {'value': array([5000.]), 'units': 'kW'}),
 ('nrel_csm_mass.sharedIndeps.rotor_diameter',
  {'value': array([126.]), 'units': 'm'}),
 ('nrel_csm_mass.blade.blade_mass',
  {'value': array([18590.66820649]), 'units': 'kg'}),
 ('nrel_csm_mass.hub.hub_mass',
  {'value': array([44078.53687493]), 'units': 'kg'}),
 ('nrel_csm_mass.pitch.pitch_system_mass',
  {'value': array([10798.90594644]), 'units': 'kg'}),
 ('nrel_csm_mass.spinner.spinner_mass',
  {'value': array([973.]), 'units': 'kg'}),
 ('nrel_csm_mass.lss.lss_mass',
  {'value': array([22820.27928238]), 'units': 'kg'}),
 ('nrel_csm_mass.bearing.main_bearing_mass',
  {'value': array([2245.41649102]), 'units': 'kg'}),
 ('nrel_csm_mass.gearbox.gearbox_mass',
  {'value': array([43468.32086769]), 'units': 'kg'}),
 ('nrel_csm_mass.hss.hss_mass', {'value': array([994.7]), 'units': 'kg'}),
 ('nrel_csm_mass.generator.generator_mass',
  {'value': array([14900.]), 'units': 'kg'}),
 ('nrel_csm_mass.b